In [1]:
#dependencies
import pandas as pd
import numpy as np

In [2]:
path_school = 'data/schools_complete.csv' #create variables for filepaths of both csv files
path_student = 'data/students_complete.csv'
school = pd.read_csv(path_school) #create dfs for both csv files
student = pd.read_csv(path_student)

In [3]:
school.head(2) #look at data

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411


In [4]:
student.head(2) #look at data

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61


# District Summary

In [5]:
number_school = school['school_name'].count() #count number of schools
number_student = school['size'].sum() #count number of students
total_budget = school['budget'].sum() #get budget for school district
reading_score_district = round(student['reading_score'].mean(), 6) #get average reading score and round to 6 digits
math_score_district = round(student['math_score'].mean(), 6) #get average math score and round to 6 digits
student['pass_reading'] = student['reading_score'] >= 70 #add column to student df to indicate if they passed reading
student['pass_math'] = student['math_score'] >= 70
student['pass_reading_and_math'] = (student['pass_reading'] == True) & (student['pass_math'] == True)
#calculate percentage of students are passing math, and round to 6 decimals
pass_math_percent = round(student[student['pass_math'] == True].count()['pass_math'] / number_student * 100, 6)
pass_reading_percent = round(student[student['pass_reading'] == True].count()['pass_reading'] / number_student *100, 6)
pass_both = round(student[student['pass_reading_and_math'] == True].count()['pass_reading_and_math'] / number_student * 100, 6)
#create dictionary for df of District Summary
district_dictionary = {'Total Schools': number_school, 'Total Students': number_student, 'Total Budget': total_budget,
                      'Average Math Score': math_score_district, 'Average Reading Score': reading_score_district, 
                      '% Passing Math': pass_math_percent, '% Passing Reading': pass_reading_percent, '% Passing Reading and Math':
                      pass_both}
#pass above dictionary into a DF
district = pd.DataFrame(district_dictionary, index=[0])
district

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Reading and Math
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary 
* Create an overview table that summarizes key metrics about each school, including:
* School Name
* School Type
* Total Students
* Total School Budget
* Per Student Budget
* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)

In [6]:
#need to combine the two dfs
student = pd.merge(student, school, how='left', on='school_name')
student.head(1)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,pass_reading,pass_math,pass_reading_and_math,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,False,True,False,0,District,2917,1910635
